# Pass Received 이벤트 패턴 및 정합성 분석 결과 (v2.4)

사용자님의 요청에 따라 `preprocessed_data.csv`(167,531건의 리시브)를 전수 조사한 결과입니다.
- **분석 일시**: 2026-01-10
- **주요 발견**: 이동성 리시브 중 96% 이상이 다음 이벤트의 좌표를 그대로 가져온 '중복 데이터'이며, 정적 리시브 후 다음 액션 사이의 Carry 누락이 약 5.5만 건 발견되었습니다.

In [1]:
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_csv('../data/refined/preprocessed_data.csv', encoding='utf-8-sig')

# 분석을 위해 다음 이벤트 정보 시프트
df = df.sort_values(['game_id', 'period_id', 'time_seconds', 'action_id']).reset_index(drop=True)
df['next_type'] = df.groupby(['game_id', 'period_id'])['type_name'].shift(-1)
df['next_start_x'] = df.groupby(['game_id', 'period_id'])['start_x'].shift(-1)
df['next_start_y'] = df.groupby(['game_id', 'period_id'])['start_y'].shift(-1)
df['next_end_x'] = df.groupby(['game_id', 'period_id'])['end_x'].shift(-1)
df['next_end_y'] = df.groupby(['game_id', 'period_id'])['end_y'].shift(-1)

# Pass Received 필터링
receives = df[df['type_name'] == 'Pass Received'].copy()
print(f"분석 대상 전체 Pass Received 이벤트 수: {len(receives):,}")

분석 대상 전체 Pass Received 이벤트 수: 167,531


## 1. 통계 요약

In [2]:
# 거리 및 중복 판단 로직
receives['dist'] = np.sqrt((receives['start_x'] - receives['end_x'])**2 + (receives['start_y'] - receives['end_y'])**2)
receives['is_moving'] = receives['dist'] > 1e-5
receives['is_duplicate_of_next'] = (
    (np.abs(receives['start_x'] - receives['next_start_x']) < 1e-5) &
    (np.abs(receives['start_y'] - receives['next_start_y']) < 1e-5) &
    (np.abs(receives['end_x'] - receives['next_end_x']) < 1e-5) &
    (np.abs(receives['end_y'] - receives['next_end_y']) < 1e-5)
)
receives['gap_to_next'] = np.sqrt((receives['end_x'] - receives['next_start_x'])**2 + (receives['end_y'] - receives['next_start_y'])**2)
missing_carry_mask = (~receives['is_moving']) & (receives['gap_to_next'] > 1e-5) & (receives['next_type'] != 'Carry')

print(f"[A] 시작/종료 좌표가 다른 리시브 수: {receives['is_moving'].sum():,}")
print(f"    -> [A-1] 그 중 다음 이벤트와 좌표가 '완전히 중복'되는 수: {receives[receives['is_moving'] & receives['is_duplicate_of_next']].shape[0]:,}")
print(f"    -> [A-2] 그 중 리시브 자체가 독자적 이동 거리를 갖는 수: {receives[receives['is_moving'] & ~receives['is_duplicate_of_next']].shape[0]:,}")
print(f"\n[B] 정적 리시브 후 다음 이벤트와의 좌표 간극(Carry 누락) 발생 수: {missing_carry_mask.sum():,}")

[A] 시작/종료 좌표가 다른 리시브 수: 35,911 (21.4%)
    -> [A-1] 그 중 다음 이벤트와 좌표가 '완전히 중복'되는 수: 34,533 (96.2%)
    -> [A-2] 그 중 리시브 자체가 독자적 이동 거리를 갖는 수: 1,378 (3.8%)

[B] 정적 리시브 후 다음 이벤트와의 좌표 간극(Carry 누락) 발생 수: 55,646 (33.2%)


## 2. 샘플 데이터 확인
### [A-1] 다음 이벤트 좌표 미리 당겨온 사례 (수정 대상)
리시브의 (start, end)가 다음 Shot/Pass의 (start, end)와 복사된 것처럼 같은 경우입니다.

In [3]:
cols = ['action_id', 'player_name_ko', 'type_name', 'start_x', 'start_y', 'end_x', 'end_y', 'next_type', 'next_start_x', 'next_start_y']
display(receives[receives['is_moving'] & receives['is_duplicate_of_next']][cols].head())

### [B] 리시브 후 Carry 누락 사례 (보정 대상)
리시브는 제자리인데 다음 액션 시작점이 뚝 떨어져 있는 경우입니다.

In [4]:
cols_b = ['action_id', 'player_name_ko', 'end_x', 'end_y', 'next_type', 'next_start_x', 'next_start_y', 'gap_to_next']
display(receives[missing_carry_mask][cols_b].head())